# SFM 2D Images to 3D Reconstruction

In [71]:
import numpy as np
import cv2
import os
import plotly.graph_objects as go
import gtsam

## IMAGE LOADING

In [2]:
def load_images_from_folder(folder_path):
    """
    Load all PNG images from the specified folder
    
    Args:
        folder_path: Path to the folder containing images
        
    Returns:
        images: List of images
        image_names: List of image filenames
    """
    images = []
    image_names = []
    
    # Get all PNG files and sort them
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.png')])
    
    for img_file in image_files:
        img_path = os.path.join(folder_path, img_file)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
            image_names.append(img_file)
            print(f"Loaded: {img_file} - Shape: {img.shape}")
    
    print(f"\nTotal images loaded: {len(images)}")
    return images, image_names


## FEATURE DETECTION & MATCHING


In [49]:
def detect_and_match_features(img1, img2, detector_type='SIFT'):
    """
    Detect features in two images and match them
    
    Args:
        img1, img2: Input images
        detector_type: 'SIFT' or 'ORB'
        
    Returns:
        pts1, pts2: Matched keypoint coordinates (Nx2 arrays)
        matches: List of good matches
    """
    # Convert to grayscale
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Feature detector
    if detector_type == 'SIFT':
        detector = cv2.SIFT_create()
    else:
        detector = cv2.ORB_create()
    
    # Detect and compute
    kp1, des1 = detector.detectAndCompute(gray1, None)
    kp2, des2 = detector.detectAndCompute(gray2, None)
    
    # Matcher
    if detector_type == 'SIFT':
        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
        matches = bf.knnMatch(des1, des2, k=2)
        
        # Lowe's ratio test
        good_matches = []
        for m, n in matches:
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)
    else:
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        good_matches = bf.match(des1, des2)
        good_matches = sorted(good_matches, key=lambda x: x.distance)
    
    # Extract matched keypoint coordinates
    pts1 = np.float32([kp1[m.queryIdx].pt for m in good_matches])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in good_matches])
    
    print(f"Found {len(good_matches)} good matches")
    
    return pts1, pts2, good_matches, kp1, kp2

## NORMALIZE POINTS

In [50]:
def normalize_points(pts):
    """
    Normalize points for better numerical stability
    Following Hartley normalization
    
    Args:
        pts: Nx2 array of points
        
    Returns:
        pts_norm: Normalized points
        T: Transformation matrix (3x3)
    """
    # Compute centroid
    centroid = np.mean(pts, axis=0)
    
    # Compute average distance from centroid
    centered = pts - centroid
    distances = np.sqrt(np.sum(centered**2, axis=1))
    avg_dist = np.mean(distances)
    
    # Avoid division by zero
    if avg_dist < np.finfo(float).eps:
        scale = 1.0
    else:
        scale = np.sqrt(2.0) / avg_dist
    
    # Create transformation matrix
    T = np.array([
        [scale, 0, -scale * centroid[0]],
        [0, scale, -scale * centroid[1]],
        [0, 0, 1]
    ], dtype=np.float64)
    
    # Normalize points
    pts_homogeneous = np.hstack([pts, np.ones((pts.shape[0], 1))])
    pts_norm_homogeneous = (T @ pts_homogeneous.T).T
    pts_norm = pts_norm_homogeneous[:, :2]
    
    return pts_norm, T

## CUSTOM FUNDAMENTAL MATRIX (8-POINT ALGORITHM)

In [51]:
def compute_fundamental_matrix_custom(pts1, pts2, method='8point'):
    """
    Compute fundamental matrix using the normalized 8-point algorithm
    
    Args:
        pts1, pts2: Matched point correspondences (Nx2)
        method: '8point' or 'ransac'
        
    Returns:
        F: Fundamental matrix (3x3)
        mask: Inlier mask (for RANSAC)
    """
    assert pts1.shape[0] >= 8, "Need at least 8 point correspondences"
    
    if method == 'ransac':
        return compute_F_ransac(pts1, pts2)
    
    # 8-point algorithm
    count = pts1.shape[0]
    
    # Get normalization matrices
    T1 = normalize_points(pts1)[1]
    T2 = normalize_points(pts2)[1]
    
    # Normalize points
    ones = np.ones((count, 1))
    pts1_h = np.hstack([pts1, ones])
    pts2_h = np.hstack([pts2, ones])
    
    pts1_norm = (T1 @ pts1_h.T).T
    pts2_norm = (T2 @ pts2_h.T).T
    
    # Build constraint matrix A (count x 9)
    A = np.zeros((count, 9), dtype=np.float64)
    for i in range(count):
        x1, y1, w1 = pts1_norm[i]
        x2, y2, w2 = pts2_norm[i]
        A[i] = [x1*x2, y1*x2, w1*x2, x1*y2, y1*y2, w1*y2, x1*w2, y1*w2, w1*w2]
    
    # Solve using SVD: A * f = 0
    U, S, Vt = np.linalg.svd(A)
    F0 = Vt[-1].reshape(3, 3)
    
    # Enforce rank-2 constraint
    U, S, Vt = np.linalg.svd(F0)
    S[2] = 0
    F0 = U @ np.diag(S) @ Vt
    
    # Denormalize: F = T2^T * F0 * T1
    F = T2.T @ F0 @ T1
    
    # Normalize so that F[2,2] = 1 (like OpenCV does)
    if abs(F[2, 2]) > np.finfo(float).eps:
        F = F / F[2, 2]
    
    return F, None

def compute_sampson_error(pt1, pt2, F):
    """
    Compute Sampson distance (symmetric epipolar distance).
    This is what OpenCV uses in RANSAC.
    
    Args:
        pt1, pt2: 2D points
        F: Fundamental matrix
        
    Returns:
        error: Sampson error
    """
    # Convert to homogeneous coordinates
    pt1_h = np.array([pt1[0], pt1[1], 1.0])
    pt2_h = np.array([pt2[0], pt2[1], 1.0])
    
    # Compute Fx1 and F^T x2
    Fx1 = F @ pt1_h
    Ftx2 = F.T @ pt2_h
    
    # x2^T F x1
    x2tFx1 = pt2_h @ F @ pt1_h
    
    # Sampson denominator
    denom = Fx1[0]**2 + Fx1[1]**2 + Ftx2[0]**2 + Ftx2[1]**2
    
    if denom < np.finfo(float).eps:
        return float('inf')
    
    # Sampson distance
    error = (x2tFx1**2) / denom
    
    return error


def check_collinear(points):
    """
    Check if points are collinear (OpenCV does this check).
    
    Args:
        points: Nx2 array of points
        
    Returns:
        bool: True if points are collinear
    """
    if len(points) < 3:
        return False
    
    # Use cross product to check collinearity
    for i in range(len(points) - 2):
        p1 = points[i]
        p2 = points[i + 1]
        p3 = points[i + 2]
        
        # Vectors
        v1 = p2 - p1
        v2 = p3 - p1
        
        # Cross product (in 2D, this is the z-component)
        cross = v1[0] * v2[1] - v1[1] * v2[0]
        
        if abs(cross) > 0.01:  # If any three points are not collinear
            return False
    
    return True


def compute_F_ransac(pts1, pts2, max_iters=2000, threshold=3.0):
    """
    Compute fundamental matrix using RANSAC
    
    Args:
        pts1, pts2: Matched points
        max_iters: Maximum RANSAC iterations
        threshold: Distance threshold for inliers
        
    Returns:
        F_best: Best fundamental matrix
        mask_best: Inlier mask
    """
    n = pts1.shape[0]
    assert n >= 8, "Need at least 8 point correspondences"
    
    best_F = None
    best_inliers = []
    best_score = 0
    sample_size = 8
    confidence = 0.99
    
    # Adaptive RANSAC
    ransac_iters = max_iters
    iter_count = 0
    
    while iter_count < ransac_iters:
        # Randomly select 8 points
        indices = np.random.choice(n, sample_size, replace=False)
        sample_pts1 = pts1[indices]
        sample_pts2 = pts2[indices]
        
        # Check for collinear points (OpenCV does this)
        if check_collinear(sample_pts1) or check_collinear(sample_pts2):
            iter_count += 1
            continue
        
        try:
            # Compute F from sample using 8-point algorithm
            F, _ = compute_fundamental_matrix_custom(sample_pts1, sample_pts2, method='8point')
            
            # Count inliers and compute score
            inliers = []
            score = 0
            for i in range(n):
                error = compute_sampson_error(pts1[i], pts2[i], F)
                if error < threshold:
                    inliers.append(i)
                    # Score based on how good the inliers are
                    score += threshold - error
            
            # Update best model
            if len(inliers) > len(best_inliers):
                best_inliers = inliers
                best_F = F
                best_score = score
                
                # Adaptive iteration count (like OpenCV)
                inlier_ratio = len(inliers) / n
                if inlier_ratio > 0.1:  # Only update if we have reasonable inliers
                    epsilon = 1.0 - inlier_ratio
                    num = np.log(1.0 - confidence)
                    denom = np.log(1.0 - (1.0 - epsilon)**sample_size)
                    if denom < 0 and denom > -100:  # Numerical stability
                        new_iters = int(np.ceil(num / denom))
                        ransac_iters = min(ransac_iters, new_iters)
        
        except:
            pass  # Skip degenerate configurations
        
        iter_count += 1
    
    # Refine F using all inliers (if we have enough)
    if len(best_inliers) >= sample_size:
        try:
            inlier_pts1 = pts1[best_inliers]
            inlier_pts2 = pts2[best_inliers]
            best_F, _ = compute_fundamental_matrix_custom(inlier_pts1, inlier_pts2, method='8point')
        except:
            pass  # Keep the current best_F if refinement fails
    
    # Create mask (same format as OpenCV)
    mask = np.zeros(n, dtype=bool)
    if best_inliers:
        mask[best_inliers] = True
    
    print(f"RANSAC: Found {len(best_inliers)}/{n} inliers")
    
    return best_F, mask

## CUSTOM ESSENTIAL MATRIX

In [52]:
def compute_essential_matrix_custom(F, K1, K2):
    """
    Compute essential matrix from fundamental matrix
    E = K2.T @ F @ K1
    
    Args:
        F: Fundamental matrix (3x3)
        K1, K2: Camera intrinsic matrices (3x3)
        
    Returns:
        E: Essential matrix (3x3)
    """
    E = K2.T @ F @ K1
    
    # Enforce essential matrix constraints (two equal singular values, one zero)
    U, S, Vt = np.linalg.svd(E)
    S = np.array([1, 1, 0])  # Ideal singular values for essential matrix
    E = U @ np.diag(S) @ Vt
    
    return E

## OPENCV FUNCTIONS (FOR COMPARISON)

In [53]:
def compute_fundamental_matrix_opencv(pts1, pts2, method='8point'):
    """
    Compute fundamental matrix using OpenCV
    
    Args:
        pts1, pts2: Matched points
        method: '8point' or 'ransac'
        
    Returns:
        F: Fundamental matrix
        mask: Inlier mask
    """
    if method == 'ransac':
        F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC, 3.0, 0.99)
    else:
        F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_8POINT)
    
    return F, mask


def compute_essential_matrix_opencv(pts1, pts2, K1, K2=None):
    """
    Compute essential matrix using OpenCV
    
    Args:
        pts1, pts2: Matched points
        K1: Camera intrinsic matrix
        K2: Camera intrinsic matrix (if None, assumes K1 == K2)
        
    Returns:
        E: Essential matrix
        mask: Inlier mask
    """
    if K2 is None:
        K2 = K1
    
    E, mask = cv2.findEssentialMat(pts1, pts2, K1, method=cv2.RANSAC, prob=0.999, threshold=1.0)
    
    return E, mask

## EVALUATION FUNCTIONS

In [54]:
def evaluate_fundamental_matrix(F, pts1, pts2):
    """
    Evaluate quality of fundamental matrix using epipolar constraint
    
    Args:
        F: Fundamental matrix
        pts1, pts2: Corresponding points
        
    Returns:
        mean_error: Mean algebraic error
        median_error: Median algebraic error
    """
    pts1_homogeneous = np.hstack([pts1, np.ones((pts1.shape[0], 1))])
    pts2_homogeneous = np.hstack([pts2, np.ones((pts2.shape[0], 1))])
    
    # Epipolar constraint: x2^T * F * x1 = 0
    errors = np.abs(np.sum(pts2_homogeneous * (F @ pts1_homogeneous.T).T, axis=1))
    
    mean_error = np.mean(errors)
    median_error = np.median(errors)
    
    return mean_error, median_error

## MAIN PROCESSING PIPELINE

In [55]:
def process_image_pairs(images, image_names, K, use_ransac=True):
    """
    Process all consecutive image pairs
    
    Args:
        images: List of images
        image_names: List of image filenames
        K: Camera intrinsic matrix
        use_ransac: Whether to use RANSAC
        
    Returns:
        results: Dictionary containing F, E matrices and matches for each pair
    """
    results = {}
    
    for i in range(len(images) - 1):
        print(f"\n{'='*60}")
        print(f"Processing pair: {image_names[i]} <-> {image_names[i+1]}")
        print(f"{'='*60}")
        
        # Detect and match features
        pts1, pts2, matches, kp1, kp2 = detect_and_match_features(images[i], images[i+1])
        
        method = 'ransac' if use_ransac else '8point'
        
        # Custom implementation
        print("\n--- CUSTOM IMPLEMENTATION ---")
        F_custom, mask_custom = compute_fundamental_matrix_custom(pts1, pts2, method=method)
        E_custom = compute_essential_matrix_custom(F_custom, K, K)
        
        mean_err_custom, median_err_custom = evaluate_fundamental_matrix(F_custom, pts1, pts2)
        print(f"Custom F matrix:\n{F_custom}")
        print(f"Mean epipolar error: {mean_err_custom:.6f}")
        print(f"Median epipolar error: {median_err_custom:.6f}")
        
        # OpenCV implementation
        print("\n--- OPENCV IMPLEMENTATION ---")
        F_opencv, mask_opencv = compute_fundamental_matrix_opencv(pts1, pts2, method=method)
        E_opencv, _ = compute_essential_matrix_opencv(pts1, pts2, K, K)
        
        mean_err_opencv, median_err_opencv = evaluate_fundamental_matrix(F_opencv, pts1, pts2)
        print(f"OpenCV F matrix:\n{F_opencv}")
        print(f"Mean epipolar error: {mean_err_opencv:.6f}")
        print(f"Median epipolar error: {median_err_opencv:.6f}")

        # Compute differences
        if not use_ransac:
            if F_opencv is not None and F_custom is not None:
                diff_8pt = np.linalg.norm(F_custom - F_opencv)
                print(f"\n8-Point Algorithm - Difference from OpenCV: {diff_8pt:.6f}")
        else:
            if F_opencv is not None and F_custom is not None:
                diff_ransac = np.linalg.norm(F_custom - F_opencv)
                print(f"RANSAC - Difference from OpenCV: {diff_ransac:.6f}")
        
        # Store results
        pair_key = f"{i}_{i+1}"
        results[pair_key] = {
            'image_names': (image_names[i], image_names[i+1]),
            'pts1': pts1,
            'pts2': pts2,
            'matches': matches,
            'F_custom': F_custom,
            'E_custom': E_custom,
            'F_opencv': F_opencv,
            'E_opencv': E_opencv,
            'mask_custom': mask_custom,
            'mask_opencv': mask_opencv
        }
        
        print(f"\n{'='*60}\n")
    
    return results

## Assinging Camera intrinsic from colmap

check this file [get_camera_intrinsics](get_camera_intrinsics.ipynb)

used colmap to get the output which has bins files inside output folder, used below given command to convert the files .txt file.

```colmap model_converter --input_path /path/to/model --output_path /path/to/output --output_type TXT```

the camera.txt file has  CAMERA_ID, MODEL, WIDTH, HEIGHT, PARAMS[]


then used the above given script to average the focal point for better estimate

In [56]:
focal_length = 1719
cx, cy = 540, 960

K = np.array([
    [focal_length, 0, cx],
    [0, focal_length, cy],
    [0, 0, 1]
], dtype=np.float64)

## Load Images

In [57]:
folder_path = './input_folder/buddha_images'
images, image_names = load_images_from_folder(folder_path)

Loaded: buddha_001.png - Shape: (1920, 1080, 3)
Loaded: buddha_002.png - Shape: (1920, 1080, 3)
Loaded: buddha_003.png - Shape: (1920, 1080, 3)
Loaded: buddha_004.png - Shape: (1920, 1080, 3)
Loaded: buddha_005.png - Shape: (1920, 1080, 3)
Loaded: buddha_006.png - Shape: (1920, 1080, 3)
Loaded: buddha_007.png - Shape: (1920, 1080, 3)
Loaded: buddha_008.png - Shape: (1920, 1080, 3)
Loaded: buddha_009.png - Shape: (1920, 1080, 3)
Loaded: buddha_010.png - Shape: (1920, 1080, 3)
Loaded: buddha_011.png - Shape: (1920, 1080, 3)
Loaded: buddha_012.png - Shape: (1920, 1080, 3)
Loaded: buddha_013.png - Shape: (1920, 1080, 3)
Loaded: buddha_014.png - Shape: (1920, 1080, 3)
Loaded: buddha_015.png - Shape: (1920, 1080, 3)
Loaded: buddha_016.png - Shape: (1920, 1080, 3)
Loaded: buddha_017.png - Shape: (1920, 1080, 3)
Loaded: buddha_018.png - Shape: (1920, 1080, 3)
Loaded: buddha_019.png - Shape: (1920, 1080, 3)
Loaded: buddha_020.png - Shape: (1920, 1080, 3)
Loaded: buddha_021.png - Shape: (1920, 1

## Process all pairs

In [58]:
np.random.seed(42)
results = process_image_pairs(images, image_names, K, use_ransac=True)


Processing pair: buddha_001.png <-> buddha_002.png
Found 344 good matches

--- CUSTOM IMPLEMENTATION ---
RANSAC: Found 300/344 inliers
Custom F matrix:
[[-7.17039053e-07 -5.27160338e-04  9.05399314e-01]
 [ 5.28032838e-04 -2.66349031e-07 -4.33780037e-01]
 [-9.05346957e-01  4.33194500e-01  1.00000000e+00]]
Mean epipolar error: 12.159070
Median epipolar error: 0.167704

--- OPENCV IMPLEMENTATION ---
OpenCV F matrix:
[[ 1.22633193e-06  1.46656977e-03 -2.34950842e+00]
 [-1.46833801e-03  3.07589073e-06  1.15672410e+00]
 [ 2.35078675e+00 -1.16198615e+00  1.00000000e+00]]
Mean epipolar error: 31.982680
Median epipolar error: 1.029104
RANSAC - Difference from OpenCV: 5.125538



Processing pair: buddha_002.png <-> buddha_003.png
Found 247 good matches

--- CUSTOM IMPLEMENTATION ---
RANSAC: Found 211/247 inliers
Custom F matrix:
[[-2.11253167e-06  3.54465452e-04  4.90420609e-02]
 [-3.56456542e-04  1.46099759e-06  5.15983824e-01]
 [-4.74408783e-02 -5.21441174e-01  1.00000000e+00]]
Mean epipolar 

### This one uses custom implementation and shows how perfect the F and E compared to open cv without ransac using only 8 point algorithm

for some reason 8 point algorithm got more reprojection errors and couldn't able to get the reconstruction while using below given functions so proceed with given ransac values alone. 

## Note
use this for verification alone for reconstruction use the above obtained using ransac

In [ ]:
results_8point = process_image_pairs(images, image_names, K, use_ransac=False)


Processing pair: buddha_001.png <-> buddha_002.png
Found 344 good matches

--- CUSTOM IMPLEMENTATION ---
Custom F matrix:
[[ 8.84834160e-06  1.47207655e-05  4.69024350e-03]
 [-1.56059867e-05  6.63705236e-07  4.64876718e-03]
 [-1.27241864e-02 -5.20901219e-03  1.00000000e+00]]
Mean epipolar error: 0.925241
Median epipolar error: 0.674021

--- OPENCV IMPLEMENTATION ---
OpenCV F matrix:
[[ 8.84832827e-06  1.47207217e-05  4.69022300e-03]
 [-1.56059405e-05  6.63703341e-07  4.64875364e-03]
 [-1.27241562e-02 -5.20899749e-03  1.00000000e+00]]
Mean epipolar error: 0.925239
Median epipolar error: 0.674020



Processing pair: buddha_002.png <-> buddha_003.png
Found 247 good matches

--- CUSTOM IMPLEMENTATION ---
Custom F matrix:
[[-2.81356288e-05 -7.98225185e-05 -8.14451066e-02]
 [ 9.22630946e-05 -2.54132445e-06 -3.95383566e-04]
 [ 9.83593840e-02 -1.30460735e-03  1.00000000e+00]]
Mean epipolar error: 2.511651
Median epipolar error: 1.446637

--- OPENCV IMPLEMENTATION ---
OpenCV F matrix:
[[-2.813

### why to use seed function in random for ransac?

I got different values everytime when i used custom ransac function given above, to avoid this i use seed in random to get similar results everytime.

#### used LLM to figure out this this is what it said

(this markdown is taken from LLM)

#### Why RANSAC Varies:

Randomness: RANSAC is inherently random - different runs produce different results
Inlier counts differ: Your implementation finds different inlier sets than OpenCV (e.g., 300 vs 303, 211 vs 219)
Local minima: RANSAC can converge to different valid solutions
Implementation details: OpenCV may use additional heuristics not visible in the source code

#### Why This Matters
RANSAC is inherently random because it:

Randomly samples 8 points each iteration
Different samples → different F matrices
Different F matrices → different inlier sets
Different inlier sets → different final results

#### why 42 as seed? what does it mean?

The number 42 is a pop culture reference and has become a tradition in programming. Here's the story:
The Meaning of 42
42 comes from "The Hitchhiker's Guide to the Galaxy" by Douglas Adams:

In the book, a supercomputer named "Deep Thought" is asked to find the "Answer to the Ultimate Question of Life, the Universe, and Everything"
After 7.5 million years of computation, it returns: 42
The joke is that they have the answer (42) but don't know what the question was!

Why Programmers Use 42

Pop culture tradition: It's a nerdy inside joke among programmers
Convention: Many code examples and tutorials use 42
Recognition: Other developers immediately understand it's arbitrary
Fun: It adds a bit of humor to technical code

## TRIANGULATION CUSTOM

In [59]:
def triangulate_points_custom(P1, P2, pts1, pts2):
    """
    Triangulate 3D points using Direct Linear Transformation (DLT)
    This matches OpenCV's cv2.triangulatePoints behavior
    
    Args:
        P1, P2: Projection matrices (3x4)
        pts1, pts2: 2D points (Nx2)
        
    Returns:
        points_3d: Triangulated 3D points (Nx3)
    """
    n_points = pts1.shape[0]
    points_4d = np.zeros((4, n_points))
    
    for i in range(n_points):
        # Build matrix A for DLT (4x4 system)
        # This is the EXACT formulation OpenCV uses
        A = np.zeros((4, 4))
        
        x1, y1 = pts1[i]
        x2, y2 = pts2[i]
        
        # First two rows from first image
        A[0] = x1 * P1[2] - P1[0]
        A[1] = y1 * P1[2] - P1[1]
        
        # Last two rows from second image  
        A[2] = x2 * P2[2] - P2[0]
        A[3] = y2 * P2[2] - P2[1]
        
        # Solve using SVD - solution is last column of V (last row of Vt)
        _, _, Vt = np.linalg.svd(A)
        X = Vt[-1]
        
        # Store as homogeneous coordinates
        points_4d[:, i] = X
    
    # Convert from homogeneous (4xN) to 3D (Nx3)
    # Normalize by w (4th coordinate)
    points_3d = (points_4d[:3] / points_4d[3]).T
    
    return points_3d

## TRIANGULATION OPENCV

In [60]:
def triangulate_points_opencv(P1, P2, pts1, pts2):
    """
    Triangulate using OpenCV for comparison
    """
    # OpenCV expects points in shape (2, N)
    pts1_t = pts1.T
    pts2_t = pts2.T
    
    # Triangulate
    points_4d = cv2.triangulatePoints(P1, P2, pts1_t, pts2_t)
    
    # Convert from homogeneous (4xN) to 3D (Nx3)
    points_3d = points_4d[:3] / points_4d[3]
    points_3d = points_3d.T
    
    return points_3d

## POSE RECOVERY FROM ESSENTIAL MATRIX

In [61]:
def recover_pose_custom(E, pts1, pts2, K):
    """
    Recover camera pose from Essential matrix
    Mimics cv2.recoverPose behavior
    
    Args:
        E: Essential matrix (3x3)
        pts1, pts2: Corresponding points (Nx2)
        K: Camera intrinsic matrix (3x3)
        
    Returns:
        R: Rotation matrix (3x3)
        t: Translation vector (3x1)
        mask: Inlier mask indicating points in front of both cameras
    """
    # Decompose Essential matrix to get 4 possible solutions
    U, _, Vt = np.linalg.svd(E)
    
    # Ensure we have a proper rotation (det = +1)
    if np.linalg.det(U @ Vt) < 0:
        Vt = -Vt
    
    # W matrix for generating rotation solutions
    W = np.array([
        [0, -1, 0],
        [1, 0, 0],
        [0, 0, 1]
    ])
    
    # Four possible solutions: (R1,t), (R1,-t), (R2,t), (R2,-t)
    R1 = U @ W @ Vt
    R2 = U @ W.T @ Vt
    t = U[:, 2].reshape(3, 1)
    
    # Make sure rotations are proper (det = 1)
    if np.linalg.det(R1) < 0:
        R1 = -R1
    if np.linalg.det(R2) < 0:
        R2 = -R2
    
    solutions = [
        (R1, t),
        (R1, -t),
        (R2, t),
        (R2, -t)
    ]
    
    # Camera 1 at origin
    P1 = K @ np.hstack([np.eye(3), np.zeros((3, 1))])
    
    best_solution = None
    max_good_points = 0
    best_mask = None
    
    # Test all 4 solutions
    for R, t_test in solutions:
        P2 = K @ np.hstack([R, t_test])
        
        # Use OpenCV triangulation here for consistency during pose recovery
        # This is what OpenCV does internally
        points_4d = cv2.triangulatePoints(P1, P2, pts1.T, pts2.T)
        points_3d = (points_4d[:3] / points_4d[3]).T
        
        # Check chirality (points must be in front of both cameras)
        # For camera 1 (at origin, looking along +Z)
        depths1 = points_3d[:, 2]
        
        # For camera 2 (transformed by R and t)
        points_cam2 = (R @ points_3d.T + t_test).T
        depths2 = points_cam2[:, 2]
        
        # Count points with positive depth in both views
        good_mask = (depths1 > 0) & (depths2 > 0)
        n_good = np.sum(good_mask)
        
        if n_good > max_good_points:
            max_good_points = n_good
            best_solution = (R, t_test)
            best_mask = good_mask
    
    R_best, t_best = best_solution
    print(f"Pose recovery: {max_good_points}/{len(pts1)} points in front of both cameras")
    
    return R_best, t_best, best_mask

## CLASS POINT CLOUD MANAGER

In [63]:
class PointCloudReconstruction:
    """
    Manages 3D reconstruction with camera poses and point cloud
    """
    
    def __init__(self, K, use_opencv=True):
        """
        Initialize reconstruction
        
        Args:
            K: Camera intrinsic matrix
            use_opencv: Use OpenCV functions (True) or custom (False)
        """
        self.K = K
        self.use_opencv = use_opencv
        
        # 3D points and their properties
        self.points_3d = []  # List of 3D points
        self.points_color = []  # RGB colors for each point
        
        # Camera poses
        self.camera_poses = []  # List of (R, t) tuples
        self.camera_ids = []  # Image indices
        
        # 2D-3D correspondences for each image
        # Maps img_idx -> list of (2d_pt, 3d_pt_idx)
        self.image_points_2d = {}  # img_idx -> list of 2D points
        self.image_points_3d_idx = {}  # img_idx -> list of 3D point indices
        
        print(f"Reconstruction initialized with {'OpenCV' if use_opencv else 'Custom'} functions")
    
    
    def add_initial_pair(self, img_idx1, img_idx2, pts1, pts2, colors1, colors2, 
                         F=None, E=None, mask=None):
        """
        Initialize reconstruction with first image pair
        
        Args:
            img_idx1, img_idx2: Image indices
            pts1, pts2: Matched points
            colors1, colors2: RGB colors at point locations
            F: Fundamental matrix
            E: Essential matrix
            mask: Inlier mask
        """
        print(f"\n{'='*60}")
        print(f"Initializing with base pair: {img_idx1} <-> {img_idx2}")
        print(f"{'='*60}")
        
        # Apply mask if provided
        if mask is not None:
            pts1 = pts1[mask]
            pts2 = pts2[mask]
            colors1 = colors1[mask]
            colors2 = colors2[mask]
        
        # Recover pose
        if self.use_opencv:
            _, R, t, pose_mask = cv2.recoverPose(E, pts1, pts2, self.K)
            pose_mask = pose_mask.ravel().astype(bool)
        else:
            R, t, pose_mask = recover_pose_custom(E, pts1, pts2, self.K)
        
        # Filter points based on pose recovery
        pts1 = pts1[pose_mask]
        pts2 = pts2[pose_mask]
        colors1 = colors1[pose_mask]
        colors2 = colors2[pose_mask]
        
        # Set up projection matrices
        # First camera at origin
        P1 = self.K @ np.hstack([np.eye(3), np.zeros((3, 1))])
        # Second camera with recovered pose
        P2 = self.K @ np.hstack([R, t])
        
        # Triangulate
        if self.use_opencv:
            points_3d = triangulate_points_opencv(P1, P2, pts1, pts2)
        else:
            points_3d = triangulate_points_custom(P1, P2, pts1, pts2)
        
        # Filter points by reprojection error and depth
        # Use more lenient threshold for custom implementation
        max_reproj_error = 5.0 if self.use_opencv else 12.0
        valid_mask = self._filter_triangulated_points(points_3d, pts1, pts2, P1, P2, 
                                                    max_reproj_error=max_reproj_error)
        
        points_3d = points_3d[valid_mask]
        pts1 = pts1[valid_mask]
        pts2 = pts2[valid_mask]
        colors1 = colors1[valid_mask]
        
        if len(points_3d) == 0:
            print("No valid points after filtering")
            return 0
        
        # Store camera poses
        self.camera_poses.append((np.eye(3), np.zeros((3, 1))))
        self.camera_poses.append((R, t))
        self.camera_ids.append(img_idx1)
        self.camera_ids.append(img_idx2)
        
        # Add 3D points
        start_idx = len(self.points_3d)
        self.points_3d.extend(points_3d)
        self.points_color.extend(colors1)
        
        # Store 2D-3D correspondences for both images
        self.image_points_2d[img_idx1] = pts1.copy()
        self.image_points_2d[img_idx2] = pts2.copy()
        self.image_points_3d_idx[img_idx1] = list(range(start_idx, start_idx + len(points_3d)))
        self.image_points_3d_idx[img_idx2] = list(range(start_idx, start_idx + len(points_3d)))
        
        print(f" Base reconstruction: {len(points_3d)} 3D points")
        print(f"  Camera 1: Identity pose")
        print(f"  Camera 2: R shape {R.shape}, t shape {t.shape}")
        
        return len(points_3d)
    
    
    def _filter_triangulated_points(self, points_3d, pts1, pts2, P1, P2, 
                                     max_reproj_error=8.0, min_depth=0.1, max_depth=100.0):
        """
        Filter triangulated points based on reprojection error and depth
        """
        N = len(points_3d)
        valid_mask = np.ones(N, dtype=bool)
        
        # Check depth in camera 1
        depths1 = points_3d[:, 2]
        valid_mask &= (depths1 > min_depth) & (depths1 < max_depth)
        
        # Check depth in camera 2
        # Extract R and t from P2
        # P2 = K @ [R | t], so we need to get [R | t] first
        K_inv = np.linalg.inv(self.K)
        Rt = K_inv @ P2
        R = Rt[:, :3]
        t = Rt[:, 3:]
        
        # Transform points to camera 2 frame
        points_cam2 = (R @ points_3d.T + t).T
        depths2 = points_cam2[:, 2]
        valid_mask &= (depths2 > min_depth) & (depths2 < max_depth)
        
        # Check reprojection error
        points_3d_hom = np.hstack([points_3d, np.ones((N, 1))]).T
        
        # Reproject to image 1
        pts1_reproj = (P1 @ points_3d_hom).T
        pts1_reproj = pts1_reproj[:, :2] / pts1_reproj[:, 2:3]
        error1 = np.linalg.norm(pts1 - pts1_reproj, axis=1)
        
        # Reproject to image 2
        pts2_reproj = (P2 @ points_3d_hom).T
        pts2_reproj = pts2_reproj[:, :2] / pts2_reproj[:, 2:3]
        error2 = np.linalg.norm(pts2 - pts2_reproj, axis=1)
        
        valid_mask &= (error1 < max_reproj_error) & (error2 < max_reproj_error)
        
        filtered_count = N - np.sum(valid_mask)
        if filtered_count > 0:
            print(f"  Filtered {filtered_count}/{N} points (reproj error or depth)")
        
        return valid_mask
    
    
    def add_incremental_view(self, img_idx, pts_2d, pts_prev, colors, 
                            prev_img_idx, F=None, E=None, mask=None):
        """
        Add a new view incrementally using PnP
        
        Args:
            img_idx: New image index
            pts_2d: 2D points in new image
            pts_prev: 2D points in previous image (matched with pts_2d)
            colors: RGB colors at point locations in new image
            prev_img_idx: Previous image index
            F, E: Fundamental and Essential matrices
            mask: Inlier mask
        """
        print(f"\nAdding incremental view: {img_idx}")
        
        # Apply mask
        if mask is not None:
            pts_2d = pts_2d[mask]
            pts_prev = pts_prev[mask]
            colors = colors[mask]
        
        # Find 2D-3D correspondences using spatial proximity matching
        if prev_img_idx not in self.image_points_2d:
            print(f"  Previous image {prev_img_idx} not in reconstruction")
            return 0
        
        prev_2d_points = self.image_points_2d[prev_img_idx]
        prev_3d_indices = self.image_points_3d_idx[prev_img_idx]
        
        # Match current points to previous reconstructed points
        # Use more lenient threshold: 10 pixels instead of 5
        match_threshold = 10.0
        
        points_2d = []
        points_3d = []
        point_colors = []
        
        for i, pt_prev in enumerate(pts_prev):
            # Find closest point in previous image's reconstructed points
            distances = np.linalg.norm(prev_2d_points - pt_prev, axis=1)
            min_dist_idx = np.argmin(distances)
            
            # Use threshold to accept match
            if distances[min_dist_idx] < match_threshold:
                pt_3d_idx = prev_3d_indices[min_dist_idx]
                points_2d.append(pts_2d[i])
                points_3d.append(self.points_3d[pt_3d_idx])
                point_colors.append(colors[i])
        
        if len(points_2d) < 6:
            print(f"  Insufficient 2D-3D correspondences: {len(points_2d)}")
            return 0
        
        points_2d = np.array(points_2d)
        points_3d = np.array(points_3d)
        point_colors = np.array(point_colors)
        
        print(f"  Found {len(points_2d)} 2D-3D correspondences")
        
        # Solve PnP to get camera pose
        if self.use_opencv:
            success, rvec, tvec, inliers = cv2.solvePnPRansac(
                points_3d, points_2d, self.K, None,
                reprojectionError=8.0, confidence=0.99
            )
            
            if not success or inliers is None or len(inliers) < 6:
                print(f"  PnP failed")
                return 0
            
            inliers = inliers.ravel()
            R, _ = cv2.Rodrigues(rvec)
            t = tvec
        else:
            # Simple PnP implementation using DLT + RANSAC
            R, t, inliers = self._solve_pnp_ransac(points_3d, points_2d)
            if R is None or len(inliers) < 6:
                print(f"  PnP failed")
                return 0
        
        print(f"  PnP: {len(inliers)}/{len(points_2d)} inliers")
        
        # Store camera pose
        self.camera_poses.append((R, t))
        self.camera_ids.append(img_idx)
        
        # Triangulate new points with previous view
        prev_cam_idx = self.camera_ids.index(prev_img_idx)
        R_prev, t_prev = self.camera_poses[prev_cam_idx]
        
        P_prev = self.K @ np.hstack([R_prev, t_prev])
        P_curr = self.K @ np.hstack([R, t])
        
        # Triangulate ALL matched points (not just those already in 3D)
        if self.use_opencv:
            new_points_3d = triangulate_points_opencv(P_prev, P_curr, pts_prev, pts_2d)
        else:
            new_points_3d = triangulate_points_custom(P_prev, P_curr, pts_prev, pts_2d)
        
        # Filter with more lenient threshold for custom
        max_reproj_error = 8.0 if self.use_opencv else 12.0
        valid_mask = self._filter_triangulated_points(
            new_points_3d, pts_prev, pts_2d, P_prev, P_curr,
            max_reproj_error=max_reproj_error
        )
        
        new_points_3d = new_points_3d[valid_mask]
        new_pts_2d = pts_2d[valid_mask]
        new_pts_prev = pts_prev[valid_mask]
        new_colors = colors[valid_mask]
        
        if len(new_points_3d) > 0:
            # Add to reconstruction
            start_idx = len(self.points_3d)
            self.points_3d.extend(new_points_3d)
            self.points_color.extend(new_colors)
            
            # Store 2D-3D correspondences for new image
            self.image_points_2d[img_idx] = new_pts_2d.copy()
            self.image_points_3d_idx[img_idx] = list(range(start_idx, start_idx + len(new_points_3d)))
            
            # Also add to previous image's correspondences (extend)
            self.image_points_2d[prev_img_idx] = np.vstack([
                self.image_points_2d[prev_img_idx], 
                new_pts_prev
            ])
            self.image_points_3d_idx[prev_img_idx].extend(
                range(start_idx, start_idx + len(new_points_3d))
            )
            
            print(f"   Added {len(new_points_3d)} new 3D points (Total: {len(self.points_3d)})")
            
            return len(new_points_3d)
        
        return 0
    
    
    def _solve_pnp_ransac(self, points_3d, points_2d, max_iters=1000, threshold=8.0):
        """
        Simple PnP RANSAC implementation
        """
        best_inliers = []
        best_R = None
        best_t = None
        N = len(points_3d)
        
        for _ in range(max_iters):
            # Sample 6 points
            if N < 6:
                return None, None, []
                
            indices = np.random.choice(N, 6, replace=False)
            
            # Solve PnP with DLT
            success, rvec, tvec = cv2.solvePnP(
                points_3d[indices], points_2d[indices], 
                self.K, None, flags=cv2.SOLVEPNP_EPNP
            )
            
            if not success:
                continue
            
            R, _ = cv2.Rodrigues(rvec)
            
            # Compute reprojection errors
            points_3d_hom = np.hstack([points_3d, np.ones((N, 1))]).T
            P = self.K @ np.hstack([R, tvec])
            
            projected = (P @ points_3d_hom).T
            projected = projected[:, :2] / projected[:, 2:3]
            
            errors = np.linalg.norm(points_2d - projected, axis=1)
            inliers = np.where(errors < threshold)[0]
            
            if len(inliers) > len(best_inliers):
                best_inliers = inliers
                best_R = R
                best_t = tvec
        
        return best_R, best_t, best_inliers
    
    
    def get_reconstruction_data(self):
        """
        Get all reconstruction data
        
        Returns:
            points_3d: Nx3 array
            colors: Nx3 array
            camera_poses: List of (R, t)
            camera_ids: List of image indices
        """
        points_3d = np.array(self.points_3d) if len(self.points_3d) > 0 else np.array([])
        colors = np.array(self.points_color) if len(self.points_color) > 0 else np.array([])
        
        return points_3d, colors, self.camera_poses, self.camera_ids

## HELPER FUNCTIONS

## Extract color from points

In [64]:
def extract_point_colors(img, pts):
    """
    Extract RGB colors at point locations
    
    Args:
        img: Image (BGR format)
        pts: Nx2 array of points
        
    Returns:
        colors: Nx3 array of RGB colors
    """
    colors = []
    h, w = img.shape[:2]
    
    for pt in pts:
        x, y = int(pt[0]), int(pt[1])
        x = max(0, min(w-1, x))
        y = max(0, min(h-1, y))
        
        # Convert BGR to RGB
        bgr = img[y, x]
        rgb = [bgr[2], bgr[1], bgr[0]]
        colors.append(rgb)
    
    return np.array(colors)

## Select best pair for base image

In [65]:
def select_best_base_pair(results):
    """
    Select the best image pair for initialization based on:
    - Number of matches
    - Epipolar geometry quality
    - Baseline (translation magnitude)
    
    Args:
        results: Dictionary from Loaded Images F and E
        
    Returns:
        best_pair_key: Key of best pair
        best_score: Score of best pair
    """
    best_score = -1
    best_pair_key = None
    
    print("\nEvaluating image pairs for base reconstruction:")
    print("-" * 60)
    
    for pair_key, data in results.items():
        pts1 = data['pts1']
        pts2 = data['pts2']
        F = data['F_opencv']
        
        # Score based on number of matches
        num_matches = len(pts1)
        
        # Score based on epipolar error
        pts1_hom = np.hstack([pts1, np.ones((len(pts1), 1))])
        pts2_hom = np.hstack([pts2, np.ones((len(pts2), 1))])
        epipolar_errors = np.abs(np.sum(pts2_hom * (F @ pts1_hom.T).T, axis=1))
        median_error = np.median(epipolar_errors)
        
        # Score based on motion (point displacement)
        displacements = np.linalg.norm(pts2 - pts1, axis=1)
        median_displacement = np.median(displacements)
        
        # Combined score (higher is better)
        # Normalize each component
        match_score = num_matches / 1000.0  # Assume max ~1000 matches
        error_score = 1.0 / (1.0 + median_error)  # Lower error is better
        motion_score = median_displacement / 100.0  # Higher motion is better (within reason)
        
        score = match_score * 0.5 + error_score * 0.3 + motion_score * 0.2
        
        print(f"Pair {pair_key}: matches={num_matches}, "
              f"median_error={median_error:.3f}, "
              f"median_disp={median_displacement:.1f}, "
              f"score={score:.3f}")
        
        if score > best_score:
            best_score = score
            best_pair_key = pair_key
    
    print("-" * 60)
    print(f"Selected base pair: {best_pair_key} (score: {best_score:.3f})")
    
    return best_pair_key, best_score

## VISUALIZATION

In [66]:
def visualize_reconstruction_3d(points_3d, colors, camera_poses, camera_ids, 
                                title="3D Reconstruction"):
    """
    Create interactive 3D visualization using Plotly
    
    Args:
        points_3d: Nx3 array of 3D points
        colors: Nx3 array of RGB colors (0-255)
        camera_poses: List of (R, t) tuples
        camera_ids: List of image indices
        title: Plot title
    """
    fig = go.Figure()
    
    # Add 3D point cloud
    if len(points_3d) > 0:
        # Normalize colors to 0-1 range for Plotly
        colors_normalized = colors / 255.0
        colors_rgb = [f'rgb({int(c[0])},{int(c[1])},{int(c[2])})' for c in colors]
        
        fig.add_trace(go.Scatter3d(
            x=points_3d[:, 0],
            y=points_3d[:, 1],
            z=points_3d[:, 2],
            mode='markers',
            marker=dict(
                size=2,
                color=colors_rgb,
                opacity=0.8
            ),
            name='3D Points',
            hovertemplate='X: %{x:.2f}<br>Y: %{y:.2f}<br>Z: %{z:.2f}<extra></extra>'
        ))
    
    # Add camera poses
    for i, (R, t) in enumerate(camera_poses):
        # Camera center in world coordinates
        C = -R.T @ t
        C = C.ravel()
        
        # Camera orientation (draw camera axes)
        axis_length = 0.5
        
        # X-axis (red)
        x_axis = R.T @ np.array([axis_length, 0, 0]).reshape(3, 1)
        fig.add_trace(go.Scatter3d(
            x=[C[0], C[0] + x_axis[0, 0]],
            y=[C[1], C[1] + x_axis[1, 0]],
            z=[C[2], C[2] + x_axis[2, 0]],
            mode='lines',
            line=dict(color='red', width=4),
            name=f'Cam {camera_ids[i]} X',
            showlegend=False,
            hovertemplate=f'Camera {camera_ids[i]}<extra></extra>'
        ))
        
        # Y-axis (green)
        y_axis = R.T @ np.array([0, axis_length, 0]).reshape(3, 1)
        fig.add_trace(go.Scatter3d(
            x=[C[0], C[0] + y_axis[0, 0]],
            y=[C[1], C[1] + y_axis[1, 0]],
            z=[C[2], C[2] + y_axis[2, 0]],
            mode='lines',
            line=dict(color='green', width=4),
            name=f'Cam {camera_ids[i]} Y',
            showlegend=False
        ))
        
        # Z-axis (blue) - viewing direction
        z_axis = R.T @ np.array([0, 0, axis_length]).reshape(3, 1)
        fig.add_trace(go.Scatter3d(
            x=[C[0], C[0] + z_axis[0, 0]],
            y=[C[1], C[1] + z_axis[1, 0]],
            z=[C[2], C[2] + z_axis[2, 0]],
            mode='lines',
            line=dict(color='blue', width=4),
            name=f'Cam {camera_ids[i]} Z',
            showlegend=False
        ))
        
        # Camera center marker
        fig.add_trace(go.Scatter3d(
            x=[C[0]],
            y=[C[1]],
            z=[C[2]],
            mode='markers+text',
            marker=dict(size=8, color='yellow', symbol='diamond'),
            text=[f'Cam {camera_ids[i]}'],
            textposition='top center',
            name=f'Camera {camera_ids[i]}',
            hovertemplate=f'Camera {camera_ids[i]}<br>X: {C[0]:.2f}<br>Y: {C[1]:.2f}<br>Z: {C[2]:.2f}<extra></extra>'
        ))
    
    # Layout
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='data',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.5)
            )
        ),
        width=1200,
        height=800,
        hovermode='closest'
    )
    
    fig.show()
    
    print(f"\n3D Visualization:")
    print(f"  Points: {len(points_3d)}")
    print(f"  Cameras: {len(camera_poses)}")
    if len(points_3d) > 0:
        print(f"  Point cloud bounds:")
        print(f"    X: [{points_3d[:, 0].min():.2f}, {points_3d[:, 0].max():.2f}]")
        print(f"    Y: [{points_3d[:, 1].min():.2f}, {points_3d[:, 1].max():.2f}]")
        print(f"    Z: [{points_3d[:, 2].min():.2f}, {points_3d[:, 2].max():.2f}]")

## MAIN RECONSTRUCTION PIPELINE

In [67]:
def run_incremental_reconstruction(images, image_names, results, K, use_opencv=True):
    """
    Run incremental 3D reconstruction
    
    Args:
        images: List of images
        image_names: List of image filenames
        results: Results from Loaded Images F and E
        K: Camera intrinsic matrix
        use_opencv: Use OpenCV functions (True) or custom (False)
        
    Returns:
        reconstruction: PointCloudReconstruction object
    """
    print("\n" + "="*70)
    print("STARTING INCREMENTAL 3D RECONSTRUCTION")
    print("="*70)
    
    # Initialize reconstruction
    reconstruction = PointCloudReconstruction(K, use_opencv=use_opencv)
    
    # Select best base pair
    base_pair_key, _ = select_best_base_pair(results)
    idx1, idx2 = map(int, base_pair_key.split('_'))
    
    # Get data for base pair
    base_data = results[base_pair_key]
    pts1 = base_data['pts1']
    pts2 = base_data['pts2']
    
    if use_opencv:
        E = base_data['E_opencv']
        mask = base_data['mask_opencv']
        if mask is not None:
            mask = mask.ravel().astype(bool)
    else:
        E = base_data['E_custom']
        mask = base_data['mask_custom']
    
    # Extract colors
    colors1 = extract_point_colors(images[idx1], pts1)
    colors2 = extract_point_colors(images[idx2], pts2)
    
    # Initialize with base pair
    num_points = reconstruction.add_initial_pair(
        idx1, idx2, pts1, pts2, colors1, colors2, 
        F=base_data['F_opencv'], E=E, mask=mask
    )
   
    if num_points == 0:
        print(" Failed to initialize reconstruction")
        return None
    
    # Track which images have been added
    reconstructed_images = {idx1, idx2}
    
    # Incrementally add remaining views
    remaining_indices = [i for i in range(len(images)) if i not in reconstructed_images]
    
    print(f"\nIncremental reconstruction:")
    print(f"  Reconstructed: {sorted(reconstructed_images)}")
    print(f"  Remaining: {remaining_indices}")
    
    iteration = 0
    max_iterations = len(images) * 2  # Prevent infinite loops
    
    while remaining_indices and iteration < max_iterations:
        iteration += 1
        added_any = False
        
        for img_idx in remaining_indices[:]:
            # Find connection to already reconstructed images
            # Try nearest reconstructed neighbors first
            potential_neighbors = []
            for recon_idx in sorted(reconstructed_images):
                pair_key = f"{min(img_idx, recon_idx)}_{max(img_idx, recon_idx)}"
                if pair_key in results:
                    distance = abs(img_idx - recon_idx)
                    potential_neighbors.append((distance, recon_idx, pair_key))
            
            # Sort by distance (prefer closer images)
            potential_neighbors.sort()
            
            for _, recon_idx, pair_key in potential_neighbors:
                pair_data = results[pair_key]
                
                # Determine which is the new image
                if img_idx < recon_idx:
                    pts_new = pair_data['pts1']
                    pts_recon = pair_data['pts2']
                else:
                    pts_new = pair_data['pts2']
                    pts_recon = pair_data['pts1']
                
                if use_opencv:
                    E = pair_data['E_opencv']
                    mask = pair_data['mask_opencv']
                    if mask is not None:
                        mask = mask.ravel().astype(bool)
                else:
                    E = pair_data['E_custom']
                    mask = pair_data['mask_custom']
                
                colors_new = extract_point_colors(images[img_idx], pts_new)
                
                # Add incremental view
                num_new_points = reconstruction.add_incremental_view(
                    img_idx, pts_new, pts_recon, colors_new, recon_idx,
                    F=pair_data['F_opencv'], E=E, mask=mask
                )
                
                if num_new_points > 0:  # Successfully added
                    reconstructed_images.add(img_idx)
                    remaining_indices.remove(img_idx)
                    added_any = True
                    break
            
            if added_any:
                break
        
        if not added_any:
            print(f"\n Cannot add remaining images: {remaining_indices}")
            print("  They may not have sufficient overlap with reconstructed views")
            break
    
    print("\n" + "="*70)
    print("RECONSTRUCTION COMPLETE")
    print("="*70)
    print(f"Final reconstruction: {len(reconstruction.points_3d)} points, "
          f"{len(reconstruction.camera_poses)} cameras")
    
    return reconstruction

## Run reconstruction with OpenCV (default)

In [68]:
print("\n Running reconstruction with OpenCV functions")
reconstruction_opencv = run_incremental_reconstruction(
    images, image_names, results, K, use_opencv=True
)

if reconstruction_opencv is not None:
    # Get reconstruction data
    points_3d, colors, camera_poses, camera_ids = reconstruction_opencv.get_reconstruction_data()
    
    # Visualize
    visualize_reconstruction_3d(points_3d, colors, camera_poses, camera_ids, 
                               title="3D Reconstruction (OpenCV)")
else:
    print(" OpenCV reconstruction failed")


 Running reconstruction with OpenCV functions

STARTING INCREMENTAL 3D RECONSTRUCTION
Reconstruction initialized with OpenCV functions

Evaluating image pairs for base reconstruction:
------------------------------------------------------------
Pair 0_1: matches=344, median_error=1.029, median_disp=1.3, score=0.322
Pair 1_2: matches=247, median_error=0.057, median_disp=9.7, score=0.427
Pair 2_3: matches=242, median_error=0.032, median_disp=43.9, score=0.500
Pair 3_4: matches=133, median_error=1.142, median_disp=79.7, score=0.366
Pair 4_5: matches=129, median_error=0.039, median_disp=47.0, score=0.447
Pair 5_6: matches=147, median_error=0.070, median_disp=13.8, score=0.382
Pair 6_7: matches=153, median_error=0.303, median_disp=18.5, score=0.344
Pair 7_8: matches=120, median_error=0.053, median_disp=75.0, score=0.495
Pair 8_9: matches=138, median_error=0.118, median_disp=28.9, score=0.395
Pair 9_10: matches=142, median_error=0.290, median_disp=36.4, score=0.376
Pair 10_11: matches=134, 


3D Visualization:
  Points: 2717
  Cameras: 24
  Point cloud bounds:
    X: [-35.58, 54.26]
    Y: [-29.35, 27.96]
    Z: [3.05, 87.81]


## Run with custom functions

In [69]:
print("\n Running reconstruction with Custom functions")
reconstruction_custom = run_incremental_reconstruction(
    images, image_names, results, K, use_opencv=False
)

if reconstruction_custom is not None:
    points_3d_custom, colors_custom, camera_poses_custom, camera_ids_custom = \
        reconstruction_custom.get_reconstruction_data()
    
    visualize_reconstruction_3d(points_3d_custom, colors_custom, 
                               camera_poses_custom, camera_ids_custom,
                               title="3D Reconstruction (Custom)")
else:
    print(" Custom reconstruction failed")


 Running reconstruction with Custom functions

STARTING INCREMENTAL 3D RECONSTRUCTION
Reconstruction initialized with Custom functions

Evaluating image pairs for base reconstruction:
------------------------------------------------------------
Pair 0_1: matches=344, median_error=1.029, median_disp=1.3, score=0.322
Pair 1_2: matches=247, median_error=0.057, median_disp=9.7, score=0.427
Pair 2_3: matches=242, median_error=0.032, median_disp=43.9, score=0.500
Pair 3_4: matches=133, median_error=1.142, median_disp=79.7, score=0.366
Pair 4_5: matches=129, median_error=0.039, median_disp=47.0, score=0.447
Pair 5_6: matches=147, median_error=0.070, median_disp=13.8, score=0.382
Pair 6_7: matches=153, median_error=0.303, median_disp=18.5, score=0.344
Pair 7_8: matches=120, median_error=0.053, median_disp=75.0, score=0.495
Pair 8_9: matches=138, median_error=0.118, median_disp=28.9, score=0.395
Pair 9_10: matches=142, median_error=0.290, median_disp=36.4, score=0.376
Pair 10_11: matches=134, 


3D Visualization:
  Points: 2637
  Cameras: 24
  Point cloud bounds:
    X: [-20.01, 40.03]
    Y: [-16.49, 36.97]
    Z: [4.68, 80.49]


## Print comparison

In [70]:
if reconstruction_opencv is not None:
    print(f"OpenCV reconstruction: {len(points_3d)} points, {len(camera_poses)} cameras")
if reconstruction_custom is not None:
    print(f"Custom reconstruction: {len(points_3d_custom)} points, {len(camera_poses_custom)} cameras")

OpenCV reconstruction: 2717 points, 24 cameras
Custom reconstruction: 2637 points, 24 cameras


# Output of colmap view in colmap gui
<video width="600" controls>
  <source src="./colmap_outputs/colmap_output.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

# Output of colmap view in meshlab
<video width="600" controls>
  <source src="./colmap_outputs/colmap_output_meshlab_view.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

# Output of OpenCV before bundle adjustment

![OUTPUT](./output_folder/3d_reconstruction_opencv_before_bundle_adjustment.png)


<video  controls>
  <source src="./output_folder/3d_reconstruction_opencv_before_bundle_adjustment.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>


# Output of Custom implementation before bundle adjustment

![OUTPUT](./output_folder/3d_reconstruction_custom_before_bundle_adjustment.png)


<video  controls>
  <source src="./output_folder/3d_reconstruction_custom_before_bundle_adjustment.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

## Bundle Adjustment using GTSAM

## CONVERT RECONSTRUCTION TO GTSAM FORMAT

### Used LLM for GTSAM implementation and error solving

In [72]:
def reconstruction_to_gtsam(reconstruction, use_robust=True):
    """
    Convert our reconstruction to GTSAM format for bundle adjustment
    
    Args:
        reconstruction: PointCloudReconstruction object
        use_robust: Use robust (Huber) loss function
        
    Returns:
        initial_estimate: Initial values for optimization
        graph: Factor graph with all constraints
        point_keys: Mapping from point index to GTSAM key
        pose_keys: Mapping from camera index to GTSAM key
    """
    print("\n" + "="*70)
    print("CONVERTING RECONSTRUCTION TO GTSAM FORMAT")
    print("="*70)
    
    # Create factor graph and initial estimate
    graph = gtsam.NonlinearFactorGraph()
    initial_estimate = gtsam.Values()
    
    # Get reconstruction data
    points_3d, colors, camera_poses, camera_ids = reconstruction.get_reconstruction_data()
    
    print(f"Input: {len(points_3d)} 3D points, {len(camera_poses)} cameras")
    
    # Create calibration object (assuming same K for all cameras)
    K_mat = reconstruction.K
    fx, fy = K_mat[0, 0], K_mat[1, 1]
    cx, cy = K_mat[0, 2], K_mat[1, 2]
    s = 0.0  # skew
    K = gtsam.Cal3_S2(fx, fy, s, cx, cy)
    
    # Noise models
    measurement_noise = gtsam.noiseModel.Isotropic.Sigma(2, 1.0)  # 1 pixel std dev
    
    if use_robust:
        # Huber loss to handle outliers
        measurement_noise = gtsam.noiseModel.Robust.Create(
            gtsam.noiseModel.mEstimator.Huber.Create(1.345),
            measurement_noise
        )
    
    # Strong prior on first camera (fix world coordinate system)
    pose_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.001, 0.001, 0.001, 0.001, 0.001, 0.001]))
    
    # Mapping from our indices to GTSAM symbol keys
    pose_keys = {}  # camera_id -> gtsam.symbol('x', i)
    point_keys = {}  # point_idx -> gtsam.symbol('l', i)
    
    # ============================
    # Add camera poses
    # ============================
    print("\nAdding camera poses...")
    for i, (R, t) in enumerate(camera_poses):
        camera_id = camera_ids[i]
        
        # Convert to GTSAM pose
        # gtsam.Pose3 uses rotation and translation
        rotation = gtsam.Rot3(R)
        translation = gtsam.Point3(t.flatten())
        pose = gtsam.Pose3(rotation, translation)
        
        # Create symbol key
        pose_key = gtsam.symbol('x', camera_id)
        pose_keys[camera_id] = pose_key
        
        # Add to initial estimate
        initial_estimate.insert(pose_key, pose)
        
        # Add strong prior on first camera to fix gauge freedom
        if i == 0:
            graph.add(gtsam.PriorFactorPose3(pose_key, pose, pose_noise))
            print(f"  Camera {camera_id}: Added with STRONG PRIOR (world origin)")
        else:
            print(f"  Camera {camera_id}: Added")
    
    # ============================
    # Add 3D points and projection factors
    # ============================
    print("\nAdding 3D points and projection factors...")
    
    num_observations = 0
    for point_idx in range(len(points_3d)):
        point_3d = points_3d[point_idx]
        
        # Create symbol key for this point
        point_key = gtsam.symbol('l', point_idx)
        point_keys[point_idx] = point_key
        
        # Add to initial estimate
        initial_estimate.insert(point_key, gtsam.Point3(point_3d))
        
        # Find all cameras that observe this point
        observations = []
        for camera_id in camera_ids:
            if camera_id in reconstruction.image_points_3d_idx:
                # Check if this point is observed in this camera
                point_indices = reconstruction.image_points_3d_idx[camera_id]
                if point_idx in point_indices:
                    # Get the corresponding 2D point
                    local_idx = point_indices.index(point_idx)
                    pt_2d = reconstruction.image_points_2d[camera_id][local_idx]
                    observations.append((camera_id, pt_2d))
        
        # Add projection factors for all observations
        for camera_id, pt_2d in observations:
            pose_key = pose_keys[camera_id]
            
            # Create projection factor
            measured = gtsam.Point2(pt_2d[0], pt_2d[1])
            factor = gtsam.GenericProjectionFactorCal3_S2(
                measured, measurement_noise, pose_key, point_key, K
            )
            graph.add(factor)
            num_observations += 1
        
        if (point_idx + 1) % 500 == 0:
            print(f"  Processed {point_idx + 1}/{len(points_3d)} points...")
    
    print(f"\n Factor graph created:")
    print(f"  Cameras: {len(camera_poses)}")
    print(f"  Points: {len(points_3d)}")
    print(f"  Observations: {num_observations}")
    print(f"  Average observations per point: {num_observations/len(points_3d):.1f}")
    
    return initial_estimate, graph, point_keys, pose_keys

## RUN BUNDLE ADJUSTMENT

In [73]:
def run_bundle_adjustment(initial_estimate, graph, optimizer_type='LM', verbose=True):
    """
    Run bundle adjustment optimization using GTSAM
    
    Args:
        initial_estimate: Initial values
        graph: Factor graph
        optimizer_type: 'LM' (Levenberg-Marquardt) or 'GN' (Gauss-Newton) or 'Dogleg'
        verbose: Print detailed progress
        
    Returns:
        result: Optimized values
    """
    print("\n" + "="*70)
    print("RUNNING BUNDLE ADJUSTMENT")
    print("="*70)
    
    # Compute initial error
    initial_error = graph.error(initial_estimate)
    num_factors = graph.size()
    
    print(f"\nInitial total error: {initial_error:.2f}")
    print(f"Number of factors: {num_factors}")
    print(f"Average error per factor: {initial_error/num_factors:.4f}")
    print(f"RMS error: {np.sqrt(initial_error/num_factors):.4f}")
    
    # Set up optimizer parameters
    max_iterations = 100
    
    if optimizer_type == 'LM':
        params = gtsam.LevenbergMarquardtParams()
        
        if verbose:
            params.setVerbosityLM("SUMMARY")  # Show iteration summary
        else:
            params.setVerbosityLM("SILENT")
            
        params.setMaxIterations(max_iterations)
        params.setRelativeErrorTol(1e-5)
        params.setAbsoluteErrorTol(1e-5)
        
        print(f"\nOptimizer: Levenberg-Marquardt")
        print(f"  Max iterations: {max_iterations}")
        print(f"  Relative error tolerance: 1e-5")
        print(f"  Absolute error tolerance: 1e-5")
        
        # Optimize
        optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate, params)
        
    elif optimizer_type == 'GN':
        params = gtsam.GaussNewtonParams()
        params.setMaxIterations(max_iterations)
        params.setRelativeErrorTol(1e-5)
        params.setAbsoluteErrorTol(1e-5)
        
        print(f"\nOptimizer: Gauss-Newton")
        print(f"  Max iterations: {max_iterations}")
        
        optimizer = gtsam.GaussNewtonOptimizer(graph, initial_estimate, params)
        
    else:  # Dogleg
        params = gtsam.DoglegParams()
        params.setMaxIterations(max_iterations)
        
        print(f"\nOptimizer: Dogleg")
        print(f"  Max iterations: {max_iterations}")
        
        optimizer = gtsam.DoglegOptimizer(graph, initial_estimate, params)
    
    print("\nOptimizing...")
    print("-" * 70)
    
    result = optimizer.optimize()
    
    # Compute final error
    final_error = graph.error(result)
    
    print("-" * 70)
    print(f"\n Optimization complete!")
    print(f"  Initial total error: {initial_error:.2f}")
    print(f"  Final total error:   {final_error:.2f}")
    print(f"  Error reduction:     {initial_error-final_error:.2f}")
    print(f"  Improvement:         {100*(initial_error-final_error)/initial_error:.2f}%")
    print(f"\n  Initial RMS error:   {np.sqrt(initial_error/num_factors):.4f}")
    print(f"  Final RMS error:     {np.sqrt(final_error/num_factors):.4f}")
    
    return result

## EXTRACT OPTIMIZED RESULTS

In [74]:
def extract_optimized_reconstruction(result, reconstruction, point_keys, pose_keys):
    """
    Extract optimized 3D points and camera poses from GTSAM result
    
    Args:
        result: Optimized GTSAM values
        reconstruction: Original PointCloudReconstruction object
        point_keys: Mapping from point index to GTSAM key
        pose_keys: Mapping from camera index to GTSAM key
        
    Returns:
        optimized_points: Nx3 array of optimized 3D points
        optimized_poses: List of (R, t) tuples for optimized camera poses
        optimized_camera_ids: List of camera IDs
    """
    print("\n" + "="*70)
    print("EXTRACTING OPTIMIZED RESULTS")
    print("="*70)
    
    # Extract optimized 3D points
    optimized_points = []
    for point_idx in sorted(point_keys.keys()):
        point_key = point_keys[point_idx]
        point = result.atPoint3(point_key)
        
        # GTSAM returns Point3 as numpy array in newer versions
        if isinstance(point, np.ndarray):
            optimized_points.append(point)
        else:
            # Older GTSAM versions return Point3 object
            optimized_points.append([point.x(), point.y(), point.z()])
    
    optimized_points = np.array(optimized_points)
    
    # Extract optimized camera poses
    optimized_poses = []
    optimized_camera_ids = []
    
    camera_ids = reconstruction.camera_ids
    for camera_id in camera_ids:
        pose_key = pose_keys[camera_id]
        pose = result.atPose3(pose_key)
        
        # Extract rotation and translation
        R = pose.rotation().matrix()
        t = pose.translation()
        
        # Handle both numpy array and Point3 object
        if isinstance(t, np.ndarray):
            t = t.reshape(3, 1)
        else:
            t = np.array([t.x(), t.y(), t.z()]).reshape(3, 1)
        
        optimized_poses.append((R, t))
        optimized_camera_ids.append(camera_id)
    
    print(f" Extracted {len(optimized_points)} optimized 3D points")
    print(f" Extracted {len(optimized_poses)} optimized camera poses")
    
    # Print statistics
    original_points = np.array(reconstruction.points_3d)
    if len(original_points) > 0 and len(optimized_points) > 0:
        # Ensure shapes match
        if optimized_points.shape == original_points.shape:
            point_changes = np.linalg.norm(optimized_points - original_points, axis=1)
            print(f"\nPoint displacement statistics:")
            print(f"  Mean:   {np.mean(point_changes):.4f} units")
            print(f"  Median: {np.median(point_changes):.4f} units")
            print(f"  Max:    {np.max(point_changes):.4f} units")
            print(f"  Min:    {np.min(point_changes):.4f} units")
        else:
            print(f"\nWarning: Shape mismatch between original and optimized points")
            print(f"  Original: {original_points.shape}")
            print(f"  Optimized: {optimized_points.shape}")
    
    return optimized_points, optimized_poses, optimized_camera_ids

## COMPUTE REPROJECTION ERRORS

In [75]:
def compute_reprojection_errors(reconstruction, points_3d, camera_poses, camera_ids):
    """
    Compute reprojection errors for all observations
    
    Args:
        reconstruction: PointCloudReconstruction object
        points_3d: Nx3 array of 3D points
        camera_poses: List of (R, t) camera poses
        camera_ids: List of camera IDs
        
    Returns:
        errors: Array of reprojection errors for each observation
    """
    K = reconstruction.K
    errors = []
    
    # Create mapping from camera_id to pose index
    camera_id_to_idx = {cam_id: i for i, cam_id in enumerate(camera_ids)}
    
    for point_idx in range(len(points_3d)):
        point_3d = points_3d[point_idx]
        
        # Find all cameras that observe this point
        for camera_id in camera_ids:
            if camera_id not in reconstruction.image_points_3d_idx:
                continue
                
            point_indices = reconstruction.image_points_3d_idx[camera_id]
            
            if point_idx in point_indices:
                # Get 2D observation
                local_idx = point_indices.index(point_idx)
                pt_2d_observed = reconstruction.image_points_2d[camera_id][local_idx]
                
                # Get camera pose
                pose_idx = camera_id_to_idx[camera_id]
                R, t = camera_poses[pose_idx]
                
                # Project 3D point to 2D
                P = K @ np.hstack([R, t])
                
                point_3d_hom = np.append(point_3d, 1.0)
                pt_2d_projected = P @ point_3d_hom
                
                # Check for valid projection
                if abs(pt_2d_projected[2]) < 1e-10:
                    continue
                    
                pt_2d_projected = pt_2d_projected[:2] / pt_2d_projected[2]
                
                # Compute error
                error = np.linalg.norm(pt_2d_observed - pt_2d_projected)
                errors.append(error)
    
    return np.array(errors)

## MAIN BUNDLE ADJUSTMENT PIPELINE

In [76]:
def perform_bundle_adjustment(reconstruction, use_robust=True, optimizer_type='LM'):
    """
    Complete bundle adjustment pipeline
    
    Args:
        reconstruction: PointCloudReconstruction object
        use_robust: Use robust (Huber) loss
        optimizer_type: Optimizer type ('LM', 'GN', or 'Dogleg')
        
    Returns:
        optimized_points: Optimized 3D points
        optimized_poses: Optimized camera poses
        optimized_camera_ids: Camera IDs
    """
    print("\n" + "="*70)
    print("BUNDLE ADJUSTMENT WITH GTSAM")
    print("="*70)
    
    # Step 1: Convert to GTSAM format
    initial_estimate, graph, point_keys, pose_keys = reconstruction_to_gtsam(
        reconstruction, use_robust=use_robust
    )
    
    # Step 2: Compute initial reprojection errors
    print("\n" + "="*70)
    print("INITIAL REPROJECTION ERRORS")
    print("="*70)
    
    original_points, _, original_poses, camera_ids = reconstruction.get_reconstruction_data()
    initial_errors = compute_reprojection_errors(
        reconstruction, original_points, original_poses, camera_ids
    )
    
    if len(initial_errors) > 0:
        print(f"\nInitial reprojection errors:")
        print(f"  Mean:   {np.mean(initial_errors):.4f} pixels")
        print(f"  Median: {np.median(initial_errors):.4f} pixels")
        print(f"  Max:    {np.max(initial_errors):.4f} pixels")
        print(f"  Std:    {np.std(initial_errors):.4f} pixels")
    else:
        print("\nWarning: No initial errors computed")
    
    # Step 3: Run bundle adjustment
    result = run_bundle_adjustment(initial_estimate, graph, optimizer_type, verbose=True)
    
    # Step 4: Extract optimized results
    optimized_points, optimized_poses, optimized_camera_ids = extract_optimized_reconstruction(
        result, reconstruction, point_keys, pose_keys
    )
    
    # Step 5: Compute final reprojection errors
    print("\n" + "="*70)
    print("FINAL REPROJECTION ERRORS")
    print("="*70)
    
    final_errors = compute_reprojection_errors(
        reconstruction, optimized_points, optimized_poses, optimized_camera_ids
    )
    
    if len(final_errors) > 0:
        print(f"\nFinal reprojection errors:")
        print(f"  Mean:   {np.mean(final_errors):.4f} pixels")
        print(f"  Median: {np.median(final_errors):.4f} pixels")
        print(f"  Max:    {np.max(final_errors):.4f} pixels")
        print(f"  Std:    {np.std(final_errors):.4f} pixels")
        
        if len(initial_errors) > 0:
            print(f"\nError reduction:")
            mean_reduction = 100*(np.mean(initial_errors)-np.mean(final_errors))/np.mean(initial_errors)
            median_reduction = 100*(np.median(initial_errors)-np.median(final_errors))/np.median(initial_errors)
            print(f"  Mean:   {mean_reduction:.1f}%")
            print(f"  Median: {median_reduction:.1f}%")
    else:
        print("\nWarning: No final errors computed")
    
    return optimized_points, optimized_poses, optimized_camera_ids

## Run bundle adjustment on OpenCV reconstruction, Convert colors to numpy array and Visualize optimized reconstruction

In [87]:
print("\n Running Bundle Adjustment on OpenCV Reconstruction")
optimized_points_opencv, optimized_poses_opencv, optimized_camera_ids_opencv = perform_bundle_adjustment(
    reconstruction_opencv, 
    use_robust=True,
    optimizer_type='LM'
)

# Convert colors to numpy array
colors_opencv = np.array(reconstruction_opencv.points_color)

# Visualize optimized reconstruction
visualize_reconstruction_3d(
    optimized_points_opencv, 
    colors_opencv, 
    optimized_poses_opencv, 
    optimized_camera_ids_opencv,
    title="3D Reconstruction - After Bundle Adjustment (OpenCV)"
)


 Running Bundle Adjustment on OpenCV Reconstruction

BUNDLE ADJUSTMENT WITH GTSAM

CONVERTING RECONSTRUCTION TO GTSAM FORMAT
Input: 2717 3D points, 24 cameras

Adding camera poses...
  Camera 11: Added with STRONG PRIOR (world origin)
  Camera 12: Added
  Camera 10: Added
  Camera 9: Added
  Camera 8: Added
  Camera 7: Added
  Camera 6: Added
  Camera 5: Added
  Camera 4: Added
  Camera 3: Added
  Camera 2: Added
  Camera 1: Added
  Camera 0: Added
  Camera 13: Added
  Camera 14: Added
  Camera 15: Added
  Camera 16: Added
  Camera 17: Added
  Camera 18: Added
  Camera 19: Added
  Camera 20: Added
  Camera 21: Added
  Camera 22: Added
  Camera 23: Added

Adding 3D points and projection factors...
  Processed 500/2717 points...
  Processed 1000/2717 points...
  Processed 1500/2717 points...
  Processed 2000/2717 points...
  Processed 2500/2717 points...

 Factor graph created:
  Cameras: 24
  Points: 2717
  Observations: 5434
  Average observations per point: 2.0

INITIAL REPROJECTION 


3D Visualization:
  Points: 2717
  Cameras: 24
  Point cloud bounds:
    X: [-253.21, 170.39]
    Y: [-94.70, 306.85]
    Z: [-225.37, 432.27]


## Run bundle adjustment on Custom reconstruction, Convert colors to numpy array and Visualize optimized reconstruction

In [85]:
# Run bundle adjustment on Custom reconstruction
print("\n Running Bundle Adjustment on Custom Reconstruction")
optimized_points_custom, optimized_poses_custom, optimized_camera_ids_custom = perform_bundle_adjustment(
    reconstruction_custom,
    use_robust=True,
    optimizer_type='LM'
)

# Convert colors to numpy array
colors_custom = np.array(reconstruction_custom.points_color)

# Visualize optimized reconstruction
visualize_reconstruction_3d(
    optimized_points_custom,
    colors_custom,
    optimized_poses_custom,
    optimized_camera_ids_custom,
    title="3D Reconstruction - After Bundle Adjustment (Custom)"
)


 Running Bundle Adjustment on Custom Reconstruction

BUNDLE ADJUSTMENT WITH GTSAM

CONVERTING RECONSTRUCTION TO GTSAM FORMAT
Input: 2637 3D points, 24 cameras

Adding camera poses...
  Camera 11: Added with STRONG PRIOR (world origin)
  Camera 12: Added
  Camera 10: Added
  Camera 9: Added
  Camera 8: Added
  Camera 7: Added
  Camera 6: Added
  Camera 5: Added
  Camera 4: Added
  Camera 3: Added
  Camera 2: Added
  Camera 1: Added
  Camera 0: Added
  Camera 13: Added
  Camera 14: Added
  Camera 15: Added
  Camera 16: Added
  Camera 17: Added
  Camera 18: Added
  Camera 19: Added
  Camera 20: Added
  Camera 21: Added
  Camera 22: Added
  Camera 23: Added

Adding 3D points and projection factors...
  Processed 500/2637 points...
  Processed 1000/2637 points...
  Processed 1500/2637 points...
  Processed 2000/2637 points...
  Processed 2500/2637 points...

 Factor graph created:
  Cameras: 24
  Points: 2637
  Observations: 5274
  Average observations per point: 2.0

INITIAL REPROJECTION 


3D Visualization:
  Points: 2637
  Cameras: 24
  Point cloud bounds:
    X: [-56.46, 144.37]
    Y: [-32.79, 108.25]
    Z: [-56.39, 210.76]


# Output of OpenCV after bundle adjustment

![OUTPUT](./output_folder/3d_reconstruction_opencv_after_bundle_adjustment.png)


<video  controls>
  <source src="./output_folder/3d_reconstruction_opencv_after_bundle_adjustment.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>


# Output of Custom implementation after bundle adjustment

![OUTPUT](./output_folder/3d_reconstruction_custom_after_bundle_adjustment.png)


<video  controls>
  <source src="./output_folder/3d_reconstruction_custom_after_bundle_adjustment.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

In [82]:
print("\nFinal reconstruction statistics:")
print(f"  OpenCV: {len(optimized_points_opencv)} points, {len(optimized_poses_opencv)} cameras")
print(f"  Custom: {len(optimized_points_custom)} points, {len(optimized_poses_custom)} cameras")


Final reconstruction statistics:
  OpenCV: 2717 points, 24 cameras
  Custom: 2637 points, 24 cameras


## Observations after bundle adjustments

The one thing that observed is the outputs is pretty good even before bundle adjustment, after bundle adjustment the output became like a dense pointcloud of previous version both custom and opencv based reconstruction almost gave similar resutls and the stats of error reduction, and points generated can be view in the above results. where as the output before BA needs to be less zoomed in to view the clear view in 3d reconstructed file but after BA all the points accumlated very closely so zoom in need to be done a lot to view the clear reconstruction